In [2]:
# import libraries
library(lubridate)
library(rjson)
library(rminer)
library(forecast)
library(writexl)


# import functions
source("functions.R")


options(repr.matrix.max.rows=2000, repr.matrix.max.cols=200)


Attaching package: 'lubridate'


The following objects are masked from 'package:base':

    date, intersect, setdiff, union


Warning message:
"package 'rminer' was built under R version 4.1.3"
Warning message:
"package 'forecast' was built under R version 4.1.3"
Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Warning message:
"package 'writexl' was built under R version 4.1.3"


In [3]:
# 0 to enable warnings
# -1 to suppress
options(warn = -1) 

In [4]:
df <- read.csv("../data/sanitizedData.csv", header = TRUE, sep=",")

In [5]:
df <- df[3:length(df)]
df

all,female,male,young,adult,weather,maxtemp,RH,maxwind,day
<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
2332,1115,1108,1122,1210,0,13,87,0,2
2801,1217,1459,1239,1562,0,14,94,45,3
2375,1168,1099,1059,1316,0,14,82,55,4
3447,1617,1651,1606,1841,1,13,78,0,5
4823,2469,2117,2318,2505,0,16,81,37,6
4978,2564,2223,2289,2689,0,16,73,0,7
2706,1192,1401,1148,1558,1,17,80,0,1
2639,1220,1308,1110,1529,1,17,70,32,2
2647,1178,1329,1152,1495,1,16,72,0,3


In [6]:
L=nrow(df)
K=7

Test = 7
Runs = 7
S= 7

# forecast:
W=(L-Test)-(Runs-1)*S 

b=1

In [6]:
getPredictionsRminer <- function(df, modelsUni, modelsMulti, target, del_cols, window, case=case, path, NPRED=7){ 
    
    if (target == "all") TS = df$all
    if (target == "male") TS = df$male
    if (target == "female") TS = df$female
    if (target == "adult") TS = df$adult
    if (target == "young") TS = df$young
    
    # rminer:
    timelags = c(1:3, 5:7) 
    D=CasesSeries(TS,timelags)
    W2=W-max(timelags) 

    YR=diff(range(TS))
    
    model_uni <- c()
    model_multi <- c()
    model_MAE <- c()
    model_NMAE <- c()
    model_RMSE <- c()
    model_RRSE <- c()
    model_ttime_uni <- c()
    model_ttime_multi <- c()
    model_ttime_total <- c()
    
    target_form <- y~.
    
    if(file.exists(path)){
        modelsUni <- fromJSON(file=path)
    } else {
        modelsUni <- gridSearch(D, modelsUni, names(D), target_form)
        exportJSON <- toJSON(modelsUni, indent=4)
        write(exportJSON, path)
    }
    
    
    for (i in 1:length(modelsUni)){
        for (l in 1:length(modelsMulti)){
            
            if (modelsUni[[i]]$rminer_best_search$mparheuristic) {
                searchh=list(search=mparheuristic(modelsUni[[i]]$rminer,modelsUni[[i]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
            } else {
                searchh=modelsUni[[i]]$rminer_best_search$algorithm
            }
            
            if(window == "none"){
                
                HD=holdout(D$y,ratio=Test,mode="order")
                
                start_time_uni = Sys.time()
            
                M1=fit(target_form, D[HD$tr,], model=modelsUni[[i]]$rminer, search = searchh)

                end_time_uni = Sys.time()
                ttime_uni <- as.numeric(difftime(end_time_uni, start_time_uni))                
                
                Pred2=lforecast(M1,D,start=(length(HD$tr)+1),Test)

                Mtrpred2=predict(M1,D[HD$tr,])

                TRSIZE=length(Mtrpred2)
                LPRED=length(Pred2)
                predColumn=c(Mtrpred2,Pred2) 

                new_D <- D$y[1:(TRSIZE+LPRED)]
                NHD=cbind(predColumn=predColumn,(df[8:(TRSIZE+LPRED+7),del_cols][2:length(df[,del_cols])]),y=new_D)
                NHD=data.frame(NHD)
                

                if (modelsMulti[[l]]$rminer_best_search$mparheuristic) {
                    searchh2=list(search=mparheuristic(modelsMulti[[l]]$rminer,modelsMulti[[l]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
                } else {
                    searchh2=modelsMulti[[l]]$rminer_best_search$algorithm
                }
                    
                start_time_multi = Sys.time()
                
                M3=fit(target_form, NHD[1:TRSIZE,], model=modelsMulti[[l]]$rminer, search = searchh2)

                end_time_multi = Sys.time()
                ttime_multi <- as.numeric(difftime(end_time_multi, start_time_multi))                
                
                predArr=predict(M3,NHD[(TRSIZE+1):(TRSIZE+LPRED),])
                                
                Y=df[HD$ts,1]
                MAE = mmetric(Y,predArr,metric="MAE")
                NMAE = mmetric(Y,predArr,metric="NMAE")
                RMSE = mmetric(Y,predArr,metric="RMSE")
                RRSE = mmetric(Y,predArr,metric="RRSE")                
                

                model_uni <- append(model_uni, modelsUni[[i]]$name)
                model_multi <- append(model_multi, modelsMulti[[l]]$name)
                model_MAE  <- append(model_MAE, MAE)
                model_NMAE <- append(model_NMAE, NMAE)
                model_RMSE <- append(model_RMSE, RMSE)
                model_RRSE <- append(model_RRSE, RRSE)
                model_ttime_uni <- append(model_ttime_uni, ttime_uni)
                model_ttime_multi <- append(model_ttime_multi, ttime_multi)
                model_ttime_total <- append(model_ttime_total, sum(ttime_uni,ttime_multi))
                
            
            } else {
                count = TRUE
            
                # vectors for saving the results
                arr_MAE=vector(length=Runs)
                arr_NMAE=vector(length=Runs)
                arr_RMSE=vector(length=Runs)
                arr_RRSE=vector(length=Runs)
                arr_ttime_uni = vector(length=Runs)
                arr_ttime_multi = vector(length=Runs)
                arr_ttime_total = vector(length=Runs)

                for(b in 1:Runs) {

                    H2=holdout(D$y,ratio=Test,mode=window,iter=b,window=W2,increment=S)
                    
                    start_time_uni = Sys.time()
                    
                    M2=fit(target_form, D[H2$tr,], model=modelsUni[[i]]$rminer, search = searchh)
                    
                    end_time_uni = Sys.time()
                    ttime_uni <- as.numeric(difftime(end_time_uni, start_time_uni))    
                    
                    Pred2=lforecast(M2,D,start=(length(H2$tr)+1),Test)

                    Mtrpred2=predict(M2,D[H2$tr,])

                    TRSIZE=length(Mtrpred2)
                    LPRED=length(Pred2)
                    predColumn=c(Mtrpred2,Pred2)  

                    new_D <- D$y[1:(TRSIZE+LPRED)]
                    HD=cbind(predColumn=predColumn,df[8:(TRSIZE+LPRED+7),del_cols][2:length(df[,del_cols])],y=new_D)
                    HD=data.frame(HD)

                    
                    if (count){
                        if (modelsMulti[[l]]$rminer_best_search$mparheuristic) {
                            searchh2=list(search=mparheuristic(modelsMulti[[l]]$rminer,modelsMulti[[l]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
                        } else {
                            searchh2=modelsMulti[[l]]$rminer_best_search$algorithm
                        }
                        count = FALSE
                    }

                    start_time_multi = Sys.time()
                    
                    M3=fit(target_form, HD[1:TRSIZE,], model=modelsMulti[[l]]$rminer, search = searchh2)

                    end_time_multi = Sys.time()
                    ttime_multi <- as.numeric(difftime(end_time_multi, start_time_multi))                     
                    
                    predArr=predict(M3,HD[(TRSIZE+1):(TRSIZE+LPRED),])

                    Y <- D[H2$ts,]$y

                    # save results
                    arr_MAE[b]=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
                    arr_NMAE[b]=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
                    arr_RMSE[b]=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
                    arr_RRSE[b]=mmetric(y=Y,x=predArr,metric="RRSE",val=YR)
                    arr_ttime_uni[b] = ttime_uni
                    arr_ttime_multi[b] = ttime_multi
                    arr_ttime_total[b] = sum(ttime_uni, ttime_multi)
                }

            # results
            MAE = mean(arr_MAE)
            NMAE = mean(arr_NMAE)
            RMSE = mean(arr_RMSE)
            RRSE = mean(arr_RRSE)
            ttime_univariate = sum(arr_ttime_uni) 
            ttime_multivariate = sum(arr_ttime_multi)     
            ttime_total = sum(arr_ttime_total)     
                   
                
            model_uni <- append(model_uni, modelsUni[[i]]$name)
            model_multi <- append(model_multi, modelsMulti[[l]]$name)
            model_MAE  <- append(model_MAE, MAE)
            model_NMAE <- append(model_NMAE, NMAE)
            model_RMSE <- append(model_RMSE, RMSE)
            model_RRSE <- append(model_RRSE, RRSE)
            model_ttime_uni <- append(model_ttime_uni, ttime_univariate)
            model_ttime_multi <- append(model_ttime_multi, ttime_multivariate)
            model_ttime_total <- append(model_ttime_total, ttime_total)     
            }   
        }                     
    }
    
        # create data frame with all the results
    results <- data.frame(model_univariate=model_uni,
                 model_multivariate = model_multi,
                 MAE=model_MAE,
                 NMAE=model_NMAE,
                 RMSE=model_RMSE,
                 RRSE=model_RRSE,
                 training_time_univariate = model_ttime_uni,
                 training_time_multivariate = model_ttime_multi,
                 training_time_total = model_ttime_total,
                 stringsAsFactors=FALSE)
}

In [15]:
getPredictionsForecast <- function(df, modelsForecast, modelsMulti, target, del_cols, window, NPRED=7){
    
    if (target == "all") TS = df$all
    if (target == "male") TS = df$male
    if (target == "female") TS = df$female
    if (target == "adult") TS = df$adult
    if (target == "young") TS = df$young
    
    
    #add <- list()
    model_uni <- c()
    model_multi <- c()
    model_MAE <- c()
    model_NMAE <- c()
    model_RMSE <- c()
    model_RRSE <- c()
    model_ttime_uni <- c()
    model_ttime_multi <- c()
    model_ttime_total <- c()
    
    YR=diff(range(TS))
    
    target_form <- y~.
    
    for (i in 1:length(modelsForecast)){
        for (l in 1:length(modelsMulti)){
            
            if(window == "none"){
                
                HD=holdout(TS,ratio=Test,mode="order")
                dtr=ts(TS[HD$tr],frequency=K)
                
                start_time_uni = Sys.time()
                
                if (modelsForecast[[i]] == "Arima") {
                    M=suppressWarnings(auto.arima(dtr)) 
                    Mtrpred = fitted(M)
                }
                if (modelsForecast[[i]] == "Holt Winters") {
                    M = suppressWarnings(HoltWinters(dtr))
                    Mtrpred = M$fitted[1:nrow(M$fitted)]
                }
                if (modelsForecast[[i]] == "NN") {
                    M=suppressWarnings(nnetar(dtr, p=7)) # P = 7, Number of seasonal lags used as inputs. (https://www.rdocumentation.org/packages/forecast/versions/8.16/topics/nnetar)
                    Mtrpred = M$fitted[8:length(M$fitted)]
                }
                if (modelsForecast[[i]] == "ETS") {
                    M=suppressWarnings(ets(dtr))
                    Mtrpred = fitted(M)
                }
                
                end_time_uni = Sys.time()
                ttime_uni <- as.numeric(difftime(end_time_uni, start_time_uni))
                
                Pred=forecast(M, h=length(HD$ts))$mean[1:Test]
                
                predColumn=c(Mtrpred, Pred)
                NHD=cbind(predColumn=predColumn, df[1:length(predColumn),del_cols][2:length(df[,del_cols])], y=TS[1:length(predColumn)])
                NHD=data.frame(NHD)
                
                if (modelsMulti[[l]]$rminer_best_search$mparheuristic) {
                    searchh2=list(search=mparheuristic(modelsMulti[[l]]$rminer,modelsMulti[[l]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
                } else {
                    searchh2=modelsMulti[[l]]$rminer_best_search$algorithm
                }
                    
                start_time_multi = Sys.time()
                
                if (modelsForecast[[i]] == "NN") {
                    M3=fit(target_form, NHD[8:length(Mtrpred),], model=modelsMulti[[l]]$rminer, search = searchh2)
                } else {
                    M3=fit(target_form, NHD[1:length(Mtrpred),], model=modelsMulti[[l]]$rminer, search = searchh2)
                }
                
                end_time_multi = Sys.time()
                ttime_multi <- as.numeric(difftime(end_time_multi, start_time_multi)) 
                
                predArr=predict(M3, NHD[(length(Mtrpred) + 1):(length(Mtrpred)+length(Pred)),])
                
                Y = TS[HD$ts]

                MAE = mmetric(Y,predArr,metric="MAE")
                NMAE = mmetric(Y,predArr,metric="NMAE")
                RMSE = mmetric(Y,predArr,metric="RMSE")
                RRSE = mmetric(Y,predArr,metric="RRSE")
                

                model_uni <- append(model_uni, modelsForecast[[i]])
                model_multi <- append(model_multi, modelsMulti[[l]]$name)
                model_MAE  <- append(model_MAE, MAE)
                model_NMAE <- append(model_NMAE, NMAE)
                model_RMSE <- append(model_RMSE, RMSE)
                model_RRSE <- append(model_RRSE, RRSE) 
                model_ttime_uni <- append(model_ttime_uni, ttime_uni)
                model_ttime_multi <- append(model_ttime_multi, ttime_multi)
                model_ttime_total <- append(model_ttime_total, sum(ttime_uni,ttime_multi))                
                
            } else {
                count = TRUE
            
                # vectors for saving the results
                arr_MAE=vector(length=Runs)
                arr_NMAE=vector(length=Runs)
                arr_RMSE=vector(length=Runs)
                arr_RRSE=vector(length=Runs)
                arr_ttime_uni = vector(length=Runs)
                arr_ttime_multi = vector(length=Runs)
                arr_ttime_total = vector(length=Runs)

                for (b in 1:Runs) {

                    H=holdout(TS,ratio=Test,mode=window,iter=b,window=W,increment=S)   
                    trinit=H$tr[1]
                    dtr=ts(TS[H$tr],frequency=K)

                    start_time_uni = Sys.time()
                    
                    if (modelsForecast[[i]] == "Arima") {
                        M=suppressWarnings(auto.arima(dtr)) 
                        Mtrpred = fitted(M)
                    }
                    if (modelsForecast[[i]] == "Holt Winters") {
                        M = suppressWarnings(HoltWinters(dtr))
                        Mtrpred = M$fitted[1:nrow(M$fitted)]
                    }
                    if (modelsForecast[[i]] == "NN") {
                        M=suppressWarnings(nnetar(dtr, p=7)) # P = 7, Number of seasonal lags used as inputs. (https://www.rdocumentation.org/packages/forecast/versions/8.16/topics/nnetar)
                        Mtrpred = M$fitted[8:length(M$fitted)]
                    }
                    if (modelsForecast[[i]] == "ETS") {
                        M=suppressWarnings(ets(dtr))
                        Mtrpred = fitted(M)
                    }

                    end_time_uni = Sys.time()
                    ttime_uni <- as.numeric(difftime(end_time_uni, start_time_uni))

                    Pred=forecast(M, h=length(H$ts))$mean[1:Test]

                    predColumn=c(Mtrpred, Pred)
                    HD=cbind(predColumn=predColumn, df[1:length(predColumn),del_cols][2:length(df[,del_cols])], y=TS[1:length(predColumn)])
                    HD=data.frame(HD)


                    if (count){
                        if (modelsMulti[[l]]$rminer_best_search$mparheuristic) {
                            searchh2=list(search=mparheuristic(modelsMulti[[l]]$rminer,modelsMulti[[l]]$rminer_best_search$algorithm),method=c("holdoutorder",2/3))
                        } else {
                            searchh2=modelsMulti[[l]]$rminer_best_search$algorithm
                        }
                        count = FALSE
                    }
                    
                    start_time_multi = Sys.time()
                    
                    if (modelsForecast[[i]] == "NN") {
                        M3=fit(target_form, HD[8:length(Mtrpred),], model=modelsMulti[[l]]$rminer, search = searchh2)
                    } else {
                        M3=fit(target_form, HD[1:length(Mtrpred),], model=modelsMulti[[l]]$rminer, search = searchh2)
                    }

                    end_time_multi = Sys.time()
                    ttime_multi <- as.numeric(difftime(end_time_multi, start_time_multi))
                    
                    predArr=predict(M3, HD[(length(Mtrpred) + 1):(length(Mtrpred)+length(Pred)),])
                    Y = TS[H$ts]

                    # save results
                    arr_MAE[b]=mmetric(y=Y,x=predArr,metric="MAE",val=YR)
                    arr_NMAE[b]=mmetric(y=Y,x=predArr,metric="NMAE",val=YR)
                    arr_RMSE[b]=mmetric(y=Y,x=predArr,metric="RMSE",val=YR)
                    arr_RRSE[b]=mmetric(y=Y,x=predArr,metric="RRSE",val=YR)  
                    arr_ttime_uni[b] = ttime_uni
                    arr_ttime_multi[b] = ttime_multi
                    arr_ttime_total[b] = sum(ttime_uni, ttime_multi)                    
                    
                }

            # results
            MAE = mean(arr_MAE)
            NMAE = mean(arr_NMAE)
            RMSE = mean(arr_RMSE)
            RRSE = mean(arr_RRSE)
            ttime_univariate = sum(arr_ttime_uni) 
            ttime_multivariate = sum(arr_ttime_multi)     
            ttime_total = sum(arr_ttime_total)   
                

            model_uni <- append(model_uni, modelsForecast[[i]])
            model_multi <- append(model_multi, modelsMulti[[l]]$name)
            model_MAE  <- append(model_MAE, MAE)
            model_NMAE <- append(model_NMAE, NMAE)
            model_RMSE <- append(model_RMSE, RMSE)
            model_RRSE <- append(model_RRSE, RRSE)   
            model_ttime_uni <- append(model_ttime_uni, ttime_univariate)
            model_ttime_multi <- append(model_ttime_multi, ttime_multivariate)
            model_ttime_total <- append(model_ttime_total, ttime_total)    

            }
        }      
    }
    # create data frame with all the results
    results <- data.frame(model_univariate=model_uni,
                 model_multivariate = model_multi,
                 MAE=model_MAE,
                 NMAE=model_NMAE,
                 RMSE=model_RMSE,
                 RRSE=model_RRSE,
                 training_time_univariate = model_ttime_uni,
                 training_time_multivariate = model_ttime_multi,
                 training_time_total = model_ttime_total,                          
                 stringsAsFactors=FALSE)
    
    results
}

In [12]:
runCasesRminerRminer <- function(target = "all", gridSearch=TRUE){
    arr_cases <- c("1","2","3", "4")
    arr_windows <- c("none","rolling", "incremental")
    results <- data.frame()
    modelsMulti <- fromJSON(file="models/rminer_models.json")
    
    for(i in 1:length(arr_cases)){
        #get case
        case <- arr_cases[i]
        case_df <- getCaseDf(df, case)
        v <- getVariables(target, case_df)
        
        if (case != "4") 
            path <- paste0("models/best_params/univariate/univariate_target=",target,"_case=1.json")
        else
            path <- paste0("models/best_params/univariate/univariate_target=",target,"_case=",case,".json")
        

        modelsUni <- fromJSON(file="models/rminer_models.json")
      
        
        pathM <- paste0("models/best_params/multivariate/multivariate_target=",target,"_case=",case,"_lags=TRUE.json")
        
        if(file.exists(pathM)){
            modelsMulti <- fromJSON(file=pathM)
        } else {
            modelsMulti <- gridSearch(case_df, modelsMulti, del_cols=v$del_cols, target_form=v$target_form)
            exportJSON <- toJSON(modelsMulti, indent=4)
            write(exportJSON, pathM)
        }    
        
        for(n in 1:length(arr_windows)){
            window <- arr_windows[n]
            r <- getPredictionsRminer(df=case_df, modelsUni=modelsUni, modelsMulti=modelsMulti ,target=target, del_cols=v$del_cols, window=window, case=case, path=path)
            r[['case']] <- case
            r[['window']] <- window
            results <- rbind(results, r)
        }
    }
    results$univariate_package <- "Rminer"
    results$multivariate_package <- "Rminer"
    col_order <- c("case", "window", "univariate_package", "multivariate_package",
               "model_univariate", "model_multivariate", "MAE", "NMAE", "RMSE", "RRSE", "training_time_total", "training_time_univariate", "training_time_multivariate")
    results <- results[, col_order]
    results <- results[with(results, order(case, window, univariate_package, model_univariate, multivariate_package, model_multivariate)), ]
    rownames(results) <- 1:nrow(results)
    write_xlsx(results,paste0("models/results/hybrid/hybrid_unipackage=rminer_multipackage=rminer_target=",target,".xlsx"))
    results
}

In [13]:
target <- "adult"
results <- runCasesRminerRminer(target=target)

[11:37:14] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:14] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[11:37:15] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg

#### All

In [27]:
results[with(results, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1024,4,none,Rminer,Rminer,Generalized Linear Model (GLM),KNN,250.9025,11.53045,416.9017,58.35375,0.048792124,0.0487921238,0.0000000000
1093,4,none,Rminer,Rminer,Support Vector Machine (KSVM),generalized linear model (GLM),270.3450,12.42394,439.2756,61.48541,0.349871159,0.2531831264,0.0966880322
1027,4,none,Rminer,Rminer,Generalized Linear Model (GLM),Multi Linear Regression,271.2608,12.46603,426.3316,59.67364,0.048326015,0.0463261604,0.0019998550
1057,4,none,Rminer,Rminer,Multilayer Percepton (MLP),Multi Linear Regression,282.0774,12.96312,452.9514,63.39963,0.364540100,0.3625409603,0.0019991398
1047,4,none,Rminer,Rminer,Multi Linear Regression (LM),Multi Linear Regression,282.0774,12.96312,452.9514,63.39963,0.003998995,0.0020000935,0.0019989014
1067,4,none,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),Multi Linear Regression,282.0774,12.96312,452.9514,63.39963,0.182620049,0.1806130409,0.0020070076
1091,4,none,Rminer,Rminer,Support Vector Machine (KSVM),Decision Three,287.4246,13.20885,442.1861,61.89280,0.311175108,0.2611751556,0.0499999523
1044,4,none,Rminer,Rminer,Multi Linear Regression (LM),KNN,309.5273,14.22460,518.6646,72.59751,0.002999783,0.0019998550,0.0009999275
1054,4,none,Rminer,Rminer,Multilayer Percepton (MLP),KNN,309.5273,14.22460,518.6646,72.59751,0.367317915,0.3663170338,0.0010008812


#### Female

In [32]:
results[with(results, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
994,4,incremental,Rminer,Rminer,Support Vector Machine (KSVM),K-Nearest Neighbor (KNN),325.8678,12.543025,384.8218,72.90773,1.64872622,1.422635078,0.22609115
1194,4,rolling,Rminer,Rminer,Support Vector Machine (KSVM),K-Nearest Neighbor (KNN),326.4657,12.566040,411.1143,77.25243,1.50006366,1.294054031,0.20600963
929,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),Random Forest,329.3971,12.678871,411.2572,76.92681,9.68998623,0.316989899,9.37299633
924,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),K-Nearest Neighbor (KNN),330.7912,12.732531,402.3474,77.17294,0.52559114,0.308603048,0.21698809
944,4,incremental,Rminer,Rminer,Multi Linear Regression (LM),K-Nearest Neighbor (KNN),333.3226,12.829969,392.0492,74.95519,0.25706744,0.014030218,0.24303722
1114,4,rolling,Rminer,Rminer,eXtreme Gradient Boosting (XGB),K-Nearest Neighbor (KNN),333.8582,12.850585,421.3377,81.96063,3.82797766,3.597900391,0.23007727
1195,4,rolling,Rminer,Rminer,Support Vector Machine (KSVM),Multi Linear Regression (LM),333.9102,12.852586,453.8239,85.15423,1.30199718,1.289967775,0.01202941
1164,4,rolling,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),K-Nearest Neighbor (KNN),334.6026,12.879240,415.9323,78.77190,2.59055996,2.375530958,0.21502900
930,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),Support Vector Machine (KSVM),335.8838,12.928553,415.8945,79.62276,1.18763709,0.320046186,0.86759090


#### Male

In [10]:
results[with(results, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1130,4,rolling,Rminer,Rminer,Generalized Linear Model (GLM),Support Vector Machine (KSVM),219.6545,10.16448,294.4150,80.65784,0.49140620,0.409412146,0.08199406
1190,4,rolling,Rminer,Rminer,Random Forest,Support Vector Machine (KSVM),224.3749,10.38292,301.0387,82.33106,0.92210913,0.859056950,0.06305218
1150,4,rolling,Rminer,Rminer,Multi Linear Regression (LM),Support Vector Machine (KSVM),225.9743,10.45693,306.7653,83.98362,0.07734609,0.011774302,0.06557178
1160,4,rolling,Rminer,Rminer,Multilayer Percepton (MLP),Support Vector Machine (KSVM),233.5035,10.80534,308.0968,84.58631,1.04899597,0.985970974,0.06302500
1170,4,rolling,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),Support Vector Machine (KSVM),235.9784,10.91987,311.5213,85.51662,0.26549411,0.204498768,0.06099534
1200,4,rolling,Rminer,Rminer,Support Vector Machine (KSVM),Support Vector Machine (KSVM),236.1948,10.92988,319.6402,87.32580,0.13424373,0.067000151,0.06724358
1125,4,rolling,Rminer,Rminer,Generalized Linear Model (GLM),Multi Linear Regression (LM),244.2348,11.30193,319.2868,86.66964,0.34796500,0.333971024,0.01399398
1120,4,rolling,Rminer,Rminer,eXtreme Gradient Boosting (XGB),Support Vector Machine (KSVM),247.4889,11.45252,326.4235,88.98629,6.69083309,6.626832962,0.06400013
1155,4,rolling,Rminer,Rminer,Multilayer Percepton (MLP),Multi Linear Regression (LM),249.2868,11.53572,331.0543,89.65824,1.01186109,0.996941090,0.01492000


#### Young

In [12]:
results[with(results, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
963,4,incremental,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),Generalized Linear Model (GLM),272.1887,12.16758,349.3327,82.43814,0.53596473,0.214898586,0.32106614
1160,4,rolling,Rminer,Rminer,Multilayer Percepton (MLP),Support Vector Machine (KSVM),281.5050,12.58404,366.5938,87.36421,1.57164812,0.194009304,1.37763882
1000,4,incremental,Rminer,Rminer,Support Vector Machine (KSVM),Support Vector Machine (KSVM),289.2239,12.92910,370.1147,87.85554,1.62265873,0.072032690,1.55062604
1140,4,rolling,Rminer,Rminer,K-Nearest Neighbor (KNN),Support Vector Machine (KSVM),295.1761,13.19518,362.3776,86.25829,1.42900681,0.005033731,1.42397308
970,4,incremental,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),Support Vector Machine (KSVM),295.7879,13.22252,368.9137,88.47146,1.69493508,0.209995031,1.48494005
964,4,incremental,Rminer,Rminer,Multilayer Percepton Ensemble (MLPE),K-Nearest Neighbor (KNN),297.1529,13.28355,373.8154,89.47438,0.41975236,0.209751129,0.21000123
1150,4,rolling,Rminer,Rminer,Multi Linear Regression (LM),Support Vector Machine (KSVM),297.4945,13.29882,380.5667,90.66967,1.41665220,0.011634588,1.40501761
923,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),Generalized Linear Model (GLM),297.8668,13.31546,378.1135,89.50287,0.66300869,0.324042320,0.33896637
1145,4,rolling,Rminer,Rminer,Multi Linear Regression (LM),Multi Linear Regression (LM),298.6887,13.35220,379.9584,90.59271,0.02300096,0.009998083,0.01300287


#### Adult

In [14]:
results[with(results, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1144,4,rolling,Rminer,Rminer,Multi Linear Regression (LM),K-Nearest Neighbor (KNN),303.2390,11.48633,397.4637,77.17969,0.21465755,0.012009144,0.20264840
1124,4,rolling,Rminer,Rminer,Generalized Linear Model (GLM),K-Nearest Neighbor (KNN),304.3129,11.52700,392.1321,76.40343,0.54403901,0.322686195,0.22135282
1194,4,rolling,Rminer,Rminer,Support Vector Machine (KSVM),K-Nearest Neighbor (KNN),310.8493,11.77460,404.4364,78.30078,1.55024695,1.338611841,0.21163511
929,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),Random Forest,314.1306,11.89889,411.3929,77.63656,1.16091490,0.315004826,0.84591007
930,4,incremental,Rminer,Rminer,Generalized Linear Model (GLM),Support Vector Machine (KSVM),314.4797,11.91211,408.2542,77.79279,0.70603061,0.322058678,0.38397193
1195,4,rolling,Rminer,Rminer,Support Vector Machine (KSVM),Multi Linear Regression (LM),314.5212,11.91368,459.6676,88.22174,1.29069901,1.276751995,0.01394701
1125,4,rolling,Rminer,Rminer,Generalized Linear Model (GLM),Multi Linear Regression (LM),315.1600,11.93788,446.6400,84.78123,0.33200836,0.318001986,0.01400638
1145,4,rolling,Rminer,Rminer,Multi Linear Regression (LM),Multi Linear Regression (LM),317.9156,12.04226,454.7063,87.17328,0.03106809,0.011001825,0.02006626
909,4,incremental,Rminer,Rminer,Decision Three,Random Forest,320.5217,12.14097,423.8767,82.84944,0.93262005,0.184026003,0.74859405


In [19]:
runCasesForecastRminer <- function(target = "all"){
    arr_cases <- c("1", "2", "3", "4")
    arr_windows <- c("none","rolling", "incremental")
    results2 <- data.frame()
    for(i in 1:length(arr_cases)){
        #get case
        case <- arr_cases[i]
        case_df <- getCaseDf(df, case)
        v <- getVariables(target, case_df)
            
        pathM <- paste0("models/best_params/multivariate/multivariate_target=",target,"_case=",case,"_lags=TRUE.json")
        
        if(file.exists(pathM)){
            modelsMulti <- fromJSON(file=pathM)
        } else {
            modelsMulti <- gridSearch(case_df, modelsMulti, del_cols=v$del_cols, target_form=v$target_form)
            exportJSON <- toJSON(modelsMulti, indent=4)
            write(exportJSON, path)
        }    
        
        for(n in 1:length(arr_windows)){
            window <- arr_windows[n]
            modelsForecast <- c("Arima", "Holt Winters", "NN", "ETS")
            r2 <- getPredictionsForecast(df=case_df, modelsForecast=modelsForecast, modelsMulti=modelsMulti, target=target, del_cols = v$del_cols, window=window)
            r2[['case']] <- case
            r2[['window']] <- window
            results2 <- rbind(results2, r2)
        }
    }
    results2$univariate_package <- "Forecast"
    results2$multivariate_package <- "Rminer"
    col_order <- c("case", "window", "univariate_package", "multivariate_package",
               "model_univariate", "model_multivariate", "MAE", "NMAE", "RMSE", "RRSE", "training_time_total", "training_time_univariate", "training_time_multivariate")
    results2 <- results2[, col_order]
    results2 <- results2[with(results2, order(case, window, univariate_package, model_univariate, multivariate_package, model_multivariate)), ]
    rownames(results2) <- 1:nrow(results2)
    write_xlsx(results2,paste0("models/results/hybrid/hybrid_unipackage=forecast_multipackage=rminer_target=",target,".xlsx"))
    results2
}

In [29]:
results2 <- runCasesForecastRminer("adult")

[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg:linear is now deprecated in favor of reg:squarederror.
[16:41:26] WARNING: amalgamation/../src/objective/regression_obj.cu:188: reg

#### All

In [21]:
results2[with(results2, order(MAE)), ][,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
290,3,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),416.9872,13.890313,520.6006,53.96063,0.386057138,0.274028063,0.1120290756
285,3,none,Forecast,Rminer,Arima,Multi Linear Regression (LM),468.9265,15.620469,723.8690,75.02954,0.272967815,0.270967007,0.0020008087
53,1,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),507.0257,16.889596,671.0628,69.55614,0.504766941,0.407249928,0.0975170135
173,2,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),507.0257,16.889596,671.0628,69.55614,0.463999987,0.415999889,0.0480000973
293,3,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),507.0257,16.889596,671.0628,69.55614,0.454361200,0.410361052,0.0440001488
300,3,none,Forecast,Rminer,ETS,Support Vector Machine (KSVM),512.3024,17.065368,666.0650,69.03812,0.543993950,0.432472944,0.1115210056
169,2,none,Forecast,Rminer,Arima,Random Forest,536.8463,17.882955,681.8093,70.67001,0.876034021,0.271031141,0.6050028801
283,3,none,Forecast,Rminer,Arima,Generalized Linear Model (GLM),538.3589,17.933340,711.6494,73.76297,0.353000879,0.305005074,0.0479958057
319,3,none,Forecast,Rminer,NN,Random Forest,540.7913,18.014366,625.4295,64.82622,2.082000256,0.353001118,1.7289991379


#### Female

In [24]:
results2[with(results2, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
417,4,none,Forecast,Rminer,ETS,Multilayer Percepton Ensemble (MLPE),196.2732,13.379222,228.3996,48.42758,0.527207851,0.489812851,0.037395000
419,4,none,Forecast,Rminer,ETS,Random Forest,222.5602,15.171109,305.1784,64.70701,2.243525982,0.501516104,1.742009878
286,3,none,Forecast,Rminer,Arima,Multilayer Percepton (MLP),229.7170,15.658962,260.5949,55.25396,1.404460907,0.231997013,1.172463894
287,3,none,Forecast,Rminer,Arima,Multilayer Percepton Ensemble (MLPE),229.8763,15.669820,260.7262,55.28181,0.675319910,0.212074995,0.463244915
412,4,none,Forecast,Rminer,ETS,eXtreme Gradient Boosting (XGB),244.0993,16.639351,327.2340,69.38346,0.933034182,0.470034122,0.463000059
290,3,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),249.0477,16.976670,277.6282,58.86552,0.339671850,0.221957922,0.117713928
300,3,none,Forecast,Rminer,ETS,Support Vector Machine (KSVM),249.8326,17.030171,338.5785,71.78884,0.566999912,0.460000038,0.106999874
53,1,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),263.2933,17.947734,315.0156,66.79279,0.507983208,0.454419136,0.053564072
304,3,none,Forecast,Rminer,Holt Winters,K-Nearest Neighbor (KNN),271.2441,18.489712,410.7226,87.08554,0.027164936,0.008167982,0.018996954


#### Male

In [26]:
results2[with(results2, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
290,3,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),167.6567,12.31864,219.2649,49.50566,0.44799995,0.326997042,0.1210029125
199,2,none,Forecast,Rminer,NN,Random Forest,187.8010,13.79875,272.4862,61.52196,1.68103194,0.324000835,1.3570311069
285,3,none,Forecast,Rminer,Arima,Multi Linear Regression (LM),203.4837,14.95104,335.0512,75.64788,0.35185003,0.348848104,0.0030019283
76,1,none,Forecast,Rminer,NN,Multilayer Percepton (MLP),210.0959,15.43687,324.0319,73.15995,1.39036918,0.370000124,1.0203690529
53,1,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),227.4648,16.71307,302.6014,68.32138,0.61100316,0.559998989,0.0510041714
293,3,none,Forecast,Rminer,ETS,Generalized Linear Model (GLM),227.4648,16.71307,302.6014,68.32138,0.56100106,0.513002157,0.0479989052
311,3,none,Forecast,Rminer,NN,Decision Three,230.7176,16.95207,284.6478,64.26781,0.38599730,0.334997177,0.0510001183
302,3,none,Forecast,Rminer,Holt Winters,eXtreme Gradient Boosting (XGB),233.7697,17.17632,301.4116,68.05274,0.50099993,0.007000923,0.4939990044
440,4,none,Forecast,Rminer,NN,Support Vector Machine (KSVM),234.0519,29.85356,283.1986,99.07835,0.28196788,0.270968914,0.0109989643


#### Young

In [28]:
results2[with(results2, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
286,3,none,Forecast,Rminer,Arima,Multilayer Percepton (MLP),196.9190,13.216037,226.8035,46.47719,1.36709690,0.221997023,1.1450998783
300,3,none,Forecast,Rminer,ETS,Support Vector Machine (KSVM),235.8552,15.829210,275.6429,56.48550,0.58097410,0.464971066,0.1160030365
285,3,none,Forecast,Rminer,Arima,Multi Linear Regression (LM),246.3074,16.530701,406.2281,83.24537,0.17400002,0.172001123,0.0019989014
290,3,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),260.8533,17.506933,368.7205,75.55921,0.29197001,0.179001093,0.1129689217
50,1,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),270.1500,18.130871,326.4905,66.90533,0.38070893,0.178984880,0.2017240524
170,2,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),270.1500,18.130871,326.4905,66.90533,0.41500092,0.184001923,0.2309989929
166,2,none,Forecast,Rminer,Arima,Multilayer Percepton (MLP),271.5506,18.224871,341.4501,69.97090,1.15416884,0.172000885,0.9821679592
46,1,none,Forecast,Rminer,Arima,Multilayer Percepton (MLP),271.5506,18.224875,341.4500,69.97087,0.55553198,0.169972897,0.3855590820
47,1,none,Forecast,Rminer,Arima,Multilayer Percepton Ensemble (MLPE),271.5522,18.224979,341.4514,69.97116,1.08699703,0.171999931,0.9149971008


#### Adult

In [30]:
results2[with(results2, order(MAE)), ][1:100,]

,case,window,univariate_package,multivariate_package,model_univariate,model_multivariate,MAE,NMAE,RMSE,RRSE,training_time_total,training_time_univariate,training_time_multivariate
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
197,2,none,Forecast,Rminer,NN,Multilayer Percepton Ensemble (MLPE),216.8167,13.766138,352.7897,72.85930,0.67515612,0.326000929,0.3491551876
304,3,none,Forecast,Rminer,Holt Winters,K-Nearest Neighbor (KNN),217.3285,13.798633,419.3044,86.59615,0.00799799,0.005996943,0.0020010471
420,4,none,Forecast,Rminer,ETS,Support Vector Machine (KSVM),221.8875,14.088098,286.5917,59.18787,0.49703097,0.427000999,0.0700299740
414,4,none,Forecast,Rminer,ETS,K-Nearest Neighbor (KNN),228.4177,14.502711,268.8496,55.52371,0.46491504,0.431915045,0.0329999924
427,4,none,Forecast,Rminer,Holt Winters,Multilayer Percepton Ensemble (MLPE),233.0497,14.796807,343.7001,70.98210,0.96097016,0.008999109,0.9519710541
426,4,none,Forecast,Rminer,Holt Winters,Multilayer Percepton (MLP),233.0605,14.797492,343.7228,70.98678,0.94654608,0.017999172,0.9285469055
425,4,none,Forecast,Rminer,Holt Winters,Multi Linear Regression (LM),233.7199,14.839361,350.0299,72.28934,0.01197219,0.009970188,0.0020020008
410,4,none,Forecast,Rminer,Arima,Support Vector Machine (KSVM),234.1977,14.869693,324.0352,66.92082,1.36199594,1.300997972,0.0609979630
404,4,none,Forecast,Rminer,Arima,K-Nearest Neighbor (KNN),243.5020,15.460447,292.4168,60.39089,1.31600189,1.283999920,0.0320019722
